<h1> A Dashboard to Show Trends in Products Using Dash </h1>
<hr>
<h3>Here we will create a dashboard to be able to quickly track, display and then analyze the sales of the products.</h3>
<br><br>
First, for this demonstration we would like to see the dashboard in the jupyter notebook inline. This is the purpose of the function below:

[(Source Link)](https://github.com/4QuantOSS/DashIntro)

In [ ]:
# A function to run the dash a app in jupyter (in an iframe)
from IPython import display
def show_app(app,
             port=9999,
             width=1150,
             height=500,
             offline=True,
             style=True,
             **dash_flask_kwargs):
    
    url = 'http://localhost:%d' % port
    iframe = '<iframe src="{url}" width={width} height={height}></iframe>'.format(url=url,
                                                                                  width=width,
                                                                                  height=height)
    display.display_html(iframe, raw=True)
    if offline:
        app.css.config.serve_locally = True
        app.scripts.config.serve_locally = True
    if style:
        external_css = ["https://fonts.googleapis.com/css?family=Raleway:400,300,600",
                        "https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css",
                        "http://getbootstrap.com/dist/css/bootstrap.min.css", ]

        for css in external_css:
            app.css.append_css({"external_url": css})

        external_js = ["https://code.jquery.com/jquery-3.2.1.min.js",
                       "https://cdn.rawgit.com/plotly/dash-app-stylesheets/a3401de132a6d0b652ba11548736b1d1e80aa10d/dash-goldman-sachs-report-js.js",
                       "http://getbootstrap.com/dist/js/bootstrap.min.js"]

        for js in external_js:
            app.scripts.append_script({"external_url": js})

    return app.run_server(debug=False, port=port, **dash_flask_kwargs)

Now we can start building our dashboard. In this dashboard, we will have a scatter plot of all the products with respect to their 'UnitPrice' and 'Quantity' and if the user hovers on each of the products, the time series plot for that product will be created on the side.

First, we have to make the layout of the dashboard and specify the objects we want there. For the time series plot, we will retrieve the hover data from the scatterplot and pass it as input and then create the plot based on this data.

In [ ]:
#importing necessary libraries
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.graph_objs as go

#importing the dataset and creating the instance
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets = external_stylesheets)

CleanDataset = r'../Cleaned-Dataset/OnlineRetail_Cleaned.csv'
Data_Cleaned = pd.read_csv(CleanDataset, index_col = 'InvoiceDate')
Data_Cleaned.index = pd.to_datetime(Data_Cleaned.index, format = '%Y-%m-%d %H:%M', box = False)

#dash layout
app.layout = html.Div(
[
    html.Div([dcc.Graph(id='crossfilter-description-scatter',hoverData={'points':[{'customdata':'HEARTS STICKERS'}]}
                        ,figure={
        'data' : [go.Scatter(x=Data_Cleaned.groupby('Description')['UnitPrice'].mean()
                           , y=Data_Cleaned.groupby('Description')['Quantity'].sum()
                           , text=Data_Cleaned.groupby('Description').sum().index
                           , customdata=Data_Cleaned.groupby('Description').sum().index
                           ,mode='markers'
                           , marker={'size': 15, 'opacity': 0.5, 'line': {'width': 0.5, 'color': 'white'}}
                            )],
        'layout': go.Layout( xaxis={'title':'UnitPrice'}, yaxis={'title':'Quantity'},
                            margin={'l': 40, 'b': 30, 't': 60, 'r': 0}, height=450, hovermode='closest',
                            annotations=[{
                'x': 0, 'y': 1.06, 'xanchor': 'left', 'yanchor': 'bottom',
                'xref': 'paper', 'yref': 'paper', 'showarrow': False,
                'align': 'left', 'bgcolor': 'rgba(255, 255, 255, 0.5)',
                'text': '<b>Quantity Vs UnitPrice Among Items</b><br>hover over a point to see time series data'
            }]
        )
    }
                        )]
    , style={'width': '45%', 'display': 'inline-block', 'padding': '0 20'}),
    
    html.Div([dcc.Graph(id='time-series')], style={'display': 'inline-block', 'width': '40%'})
]
)

#a function to get the data and create the time series
def create_time_series(dff, title):
    return {
        'data': [go.Scatter(
            x=dff.index,
            y=dff.values,
            mode='lines+markers'
        )],
        'layout': {
            'height': 400,
            'margin': {'l': 40, 'b': 65, 'r': 35, 't': 40},
            'annotations': [{
                'x': 0, 'y': 1, 'xanchor': 'left', 'yanchor': 'bottom',
                'xref': 'paper', 'yref': 'paper', 'showarrow': False,
                'align': 'left', 'bgcolor': 'rgba(255, 255, 255, 0.5)',
                'text': title
            }],
            'xaxis': {'showgrid': False}
        }
    }

#passing the hoverdata as input and creating the times series plot
@app.callback(
               dash.dependencies.Output('time-series','figure'),
               [ dash.dependencies.Input('crossfilter-description-scatter','hoverData')]
)
def time_series(hoverData):
    item_name = hoverData['points'][0]['customdata']
    dff = Data_Cleaned[Data_Cleaned['Description'] == item_name]
    dff = dff.groupby('InvoiceMonth', sort=False)['FinalPrice'].sum()
    title = '<b>{}</b><br>{}'.format(item_name, 'Monthly Sales, Pounds')
    return create_time_series(dff, title)

show_app(app)


<h3> Unfortunately due to limitations, it's not possible to render dash's interactive dashboards on github, so here is an example of what happens and what the resulting dashboard will look like after running the script: <h3>
<br>
    
![Dashboard](Dashboard.gif)